In [ ]:
# load the trends data from csv
import pandas as pd

trends = pd.read_csv('trends_raw.csv', header=None, names=['wf_id', 'trend'])
#trends = pd.read_csv('trends.csv', header=1, names=['wf_id', 'trend', 'names']).drop(columns=['names'])

# trends = trends.dropna() # there should not be any

trends.describe()

In [ ]:
# Create a mapping between wf_id and farm names
import pandas as pd
from src.DatabaseGetInfo import DatabaseAnalyzer

# Initialize the database analyzer
path = '/home/wheatley/WFD/wfd.db'  # Database path
analyzer = DatabaseAnalyzer.WindFarmAnalyzer(path)

# Create a dictionary to store wf_id to farm name mapping
wf_id_to_name = {}

# Get farm names for each wf_id in trends
for wf_id in trends['wf_id']:
    try:
        farm_info = analyzer.get_farm_info(wf_id)
        if farm_info:
            farm_name = farm_info.get('plant_name')
            if farm_name:
                wf_id_to_name[wf_id] = farm_name
            else:
                # If no specific name field, try to construct from available fields
                wf_id_to_name[wf_id] = f"Farm_{wf_id}"
        else:
            wf_id_to_name[wf_id] = f"Unknown_Farm_{wf_id}"
    except Exception as e:
        print(f"Error getting farm info for wf_id {wf_id}: {e}")
        wf_id_to_name[wf_id] = f"Error_Farm_{wf_id}"

# Add farm names to the trends DataFrame
trends['farm_name'] = trends['wf_id'].map(wf_id_to_name)

trends

In [ ]:
# mean trend visualization
import matplotlib.pyplot as plt
import numpy as np

# get the mean of the trends
mean_trend = np.mean(trends['trend'].values)
# get the std of the trends
std_trend = np.std(trends['trend'].values)
# get the max of the trends

# visualize the trends
plt.figure(figsize=(20, 10), dpi=100)
plt.axhline(y=mean_trend, color='black', linestyle='-', label=f'Mean = {mean_trend:.2f}')
plt.axhline(y=mean_trend+std_trend, color='g', linestyle='--', linewidth=1, label=f'1 Std = {std_trend:.2f}')
plt.axhline(y=mean_trend-std_trend, color='g', linestyle='--', linewidth=1) # No label needed if it's the same category
plt.axhline(y=mean_trend+std_trend*2, color='y', linestyle='--', linewidth=3, label=f'2 Std = {std_trend*2:.2f}')
plt.axhline(y=mean_trend-std_trend*2, color='y', linestyle='--', linewidth=3) # No label needed
plt.axhline(y=mean_trend+std_trend*3, color='r', linestyle='--', linewidth=1, label=f'3 Std = {std_trend*3:.2f}')
plt.axhline(y=mean_trend-std_trend*3, color='r', linestyle='--', linewidth=1) # No label needed
plt.plot(trends['wf_id'], trends['trend'], 'o', markersize=8, label='Wind Farm Trends') # Added a label for the scatter plot
plt.xlabel('Wind Farm ID')
plt.ylabel('Trend')

plt.legend(loc='upper left', ncol=1, frameon=True)

plt.title('Wind Farm Trends')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Assuming 'trends' DataFrame is already defined and contains 'trend' column
# Example 'trends' DataFrame for testing purposes:
# np.random.seed(42)
# trends = pd.DataFrame({'trend': np.random.normal(loc=50, scale=10, size=100)})
# trends.loc[0, 'trend'] = 10 # Adding an outlier
# trends.loc[1, 'trend'] = 90 # Adding another outlier

# 1. Calculate Q1, Q3, and IQR for the 'trend' data
Q1 = trends['trend'].quantile(0.25)
Q3 = trends['trend'].quantile(0.75)
IQR = Q3 - Q1
median = trends['trend'].median() # Calculate median
mean = trends['trend'].mean() # Calculate mean

# Calculate whisker values
# Lower whisker is Q1 - 1.5*IQR or the lowest data point within that range
lower_whisker = trends['trend'][trends['trend'] >= (Q1 - 1.5 * IQR)].min()
# Upper whisker is Q3 + 1.5*IQR or the highest data point within that range
upper_whisker = trends['trend'][trends['trend'] <= (Q3 + 1.5 * IQR)].max()

print(f"First Quartile (Q1): {Q1:.2f}")
print(f"Third Quartile (Q3): {Q3:.2f}")
print(f"Interquartile Range (IQR): {IQR:.2f}")
print(f"Median: {median:.2f}") # Print median
print(f"Mean: {mean:.2f}") # Print mean
print(f"Lower Whisker: {lower_whisker:.2f}")
print(f"Upper Whisker: {upper_whisker:.2f}")

# 2. Create the figure and two subplots side-by-side
fig, axes = plt.subplots(1, 2, figsize=(12, 8), dpi=100, sharey=True) # sharey=True ensures same y-axis scale

# Plot data points on the left subplot (axes[0])
sns.stripplot(y=trends['trend'], jitter=0.2, color="darkblue", alpha=0.7, ax=axes[0])
axes[0].set_ylabel('Trend Value')
axes[0].set_title('Individual Data Points')
axes[0].set_xticks([]) # Remove x-axis tick labels for the stripplot
axes[0].grid(True, axis='y', linestyle='--', alpha=0.7)

# Plot box plot on the right subplot (axes[1])
sns.boxplot(y=trends['trend'], color='lightblue', showmeans=True,
            meanprops={"marker":"o", "markerfacecolor":"red", "markeredgecolor":"black", "markersize":"8"},
            medianprops={'color': 'orange', 'linewidth': 2},
            ax=axes[1])
axes[1].set_ylabel('') # No y-label for the second plot since y-axis is shared
axes[1].set_title('Box Plot of Trends')
axes[1].set_xticks([]) # Remove x-axis tick labels for the box plot
axes[1].grid(True, axis='y', linestyle='--', alpha=0.7)


# Overall title for the figure
fig.suptitle('Individual Data Points & Box Plot', fontsize=16)

# Create a custom legend for the box plot elements (mean, median, whiskers) and stripplot points
from matplotlib.lines import Line2D
legend_elements_box = [
    Line2D([0], [0], color='orange', lw=2, label=f'Median: {median:.2f}'),
    Line2D([0], [0], marker='o', color='w', label=f'Mean: {mean:.2f}',
           markerfacecolor='red', markeredgecolor='black', markersize=8),
    Line2D([0], [0], color='blue', lw=1, linestyle='--', label=f'Lower Whisker: {lower_whisker:.2f}'),
    Line2D([0], [0], color='blue', lw=1, linestyle='--', label=f'Upper Whisker: {upper_whisker:.2f}'),
]

# You can place the legend on one of the subplots or outside the figure
axes[1].legend(handles=legend_elements_box, loc='upper left', bbox_to_anchor=(0, 1)) # Place outside for clarity
axes[0].legend(handles=[Line2D([0], [0], marker='o', color='darkblue', lw=0, label='Individual Data Points',
           markerfacecolor='darkblue', markeredgecolor='darkblue', markersize=6)],
               loc='upper left', bbox_to_anchor=(0, 1))

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
plt.show()

# 3. (Optional) Programmatically identify and list the outliers
# These are already calculated as part of the whisker calculation
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = trends[(trends['trend'] < lower_bound) | (trends['trend'] > upper_bound)]

if not outliers.empty:
    print("\nOutliers identified based on the 1.5xIQR rule:")
    print(outliers)
else:
    print("\nNo outliers found by the 1.5xIQR rule outside the whiskers.")

In [ ]:
# remove the outliers from the trends dataframe
trends = trends[(trends['trend'] >= lower_bound) & (trends['trend'] <= upper_bound)]
trends.to_csv('trends.csv', index=False)
trends['trend'].describe().drop(labels=['count'])

In [ ]:
# mean trend visualization
import matplotlib.pyplot as plt
import numpy as np

# get the mean of the trends
mean_trend = np.mean(trends['trend'].values)
# get the std of the trends
std_trend = np.std(trends['trend'].values)
# get the max of the trends

# visualize the trends
plt.figure(figsize=(20, 10), dpi=100)
plt.axhline(y=mean_trend, color='black', linestyle='-', label=f'Mean = {mean_trend:.2f}')
plt.axhline(y=mean_trend+std_trend, color='g', linestyle='--', linewidth=1, label=f'1 Std = {std_trend:.2f}')
plt.axhline(y=mean_trend-std_trend, color='g', linestyle='--', linewidth=1) # No label needed if it's the same category
plt.axhline(y=mean_trend+std_trend*2, color='y', linestyle='--', linewidth=3, label=f'2 Std = {std_trend*2:.2f}')
plt.axhline(y=mean_trend-std_trend*2, color='y', linestyle='--', linewidth=3) # No label needed
plt.axhline(y=mean_trend+std_trend*3, color='r', linestyle='--', linewidth=1, label=f'3 Std = {std_trend*3:.2f}')
plt.axhline(y=mean_trend-std_trend*3, color='r', linestyle='--', linewidth=1) # No label needed
plt.plot(trends['wf_id'], trends['trend'], 'o', markersize=8, label='Wind Farm Trends') # Added a label for the scatter plot
plt.xlabel('Wind Farm ID')
plt.ylabel('Trend')

plt.legend(loc='upper left', ncol=1, frameon=True)

plt.title('Wind Farm Trends')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 1. Calculate Q1, Q3, and IQR for the 'trend' data
Q1 = trends['trend'].quantile(0.25)
Q3 = trends['trend'].quantile(0.75)
IQR = Q3 - Q1
mean = trends['trend'].mean()

print(f"First Quartile (Q1): {Q1:.2f}")
print(f"Third Quartile (Q3): {Q3:.2f}")
print(f"Interquartile Range (IQR): {IQR:.2f}")
print(f"Mean: {mean:.2f}")

# 2. Create the figure and two subplots side-by-side
fig, axes = plt.subplots(1, 2, figsize=(12, 8), dpi=100, sharey=True) # sharey=True ensures same y-axis scale

# Plot data points on the left subplot (axes[0])
sns.stripplot(y=trends['trend'], jitter=0.2, color="darkblue", alpha=0.7, ax=axes[0])
axes[0].set_ylabel('Trend Value')
axes[0].set_title('Individual Data Points')
axes[0].set_xticks([]) # Remove x-axis tick labels for the stripplot
axes[0].grid(True, axis='y', linestyle='--', alpha=0.7)

# Plot box plot on the right subplot (axes[1])
sns.boxplot(y=trends['trend'], color='lightblue', showmeans=True,
            meanprops={"marker":"o", "markerfacecolor":"red", "markeredgecolor":"black", "markersize":"8"},
            medianprops={'color': 'orange', 'linewidth': 2},
            ax=axes[1])
axes[1].set_ylabel('') # No y-label for the second plot since y-axis is shared
axes[1].set_title('Box Plot of Trends')
axes[1].set_xticks([]) # Remove x-axis tick labels for the box plot
axes[1].grid(True, axis='y', linestyle='--', alpha=0.7)


# Overall title for the figure
fig.suptitle('Individual Data Points & Box Plot', fontsize=16)

# Create a custom legend for the box plot elements (mean, median) and stripplot points
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='orange', lw=2, label='Median'),
    Line2D([0], [0], marker='o', color='w', label='Mean',
           markerfacecolor='red', markeredgecolor='black', markersize=8),
]
# You can place the legend on one of the subplots or outside the figure
axes[1].legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1, 1)) # Place outside for clarity
axes[0].legend(handles=[Line2D([0], [0], marker='o', color='darkblue', lw=0, label='Individual Data Points',
           markerfacecolor='darkblue', markeredgecolor='darkblue', markersize=6)],
               loc='upper left', bbox_to_anchor=(0, 1))

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
plt.show()

# 3. (Optional) Programmatically identify and list the outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = trends[(trends['trend'] < lower_bound) | (trends['trend'] > upper_bound)]

if not outliers.empty:
    print("\nOutliers identified based on the 1.5xIQR rule:")
    print(outliers)
else:
    print("\nNo outliers found by the 1.5xIQR rule outside the whiskers.")

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

alpha = 0.05

# Choose the type of test: 'one-tailed' or 'two-tailed'.
test_type = 'one-tailed'

# Use the 'trend' column from your DataFrame
trends_data = trends['trend'].values

# --- Statistical Analysis ---
if len(trends_data) > 0:
    # Perform a one-sample t-test
    t_statistic, p_value_two_tailed = stats.ttest_1samp(a=trends_data, popmean=0)

    # --- Adjust P-value Based on Test Type ---
    if test_type == 'one-tailed':
        if t_statistic < 0:
            p_value = p_value_two_tailed / 2
        else:
            p_value = 1 - p_value_two_tailed / 2
    elif test_type == 'two-tailed':
        p_value = p_value_two_tailed
    else:
        raise ValueError("Invalid 'test_type'. Please choose either 'one-tailed' or 'two-tailed'.")

    # --- Results and Interpretation ---
    mean_trend = np.mean(trends_data)
    std_dev = np.std(trends_data, ddof=1)

    print("\n--- Statistical Test Results ---")
    print(f"Analysis based on the trends of {len(trends_data)} wind farms.")
    print(f"Mean Trend: {mean_trend:.4f}% per year")
    print(f"Standard Deviation of Trends: {std_dev:.4f}")
    print(f"T-statistic: {t_statistic:.4f}")
    print(f"Significance Level (alpha): {alpha}")
    print(f"Test Type: {test_type}")
    print(f"Calculated P-value: {p_value:.6f}")

    print("\n--- Conclusion ---")
    if p_value < alpha:
        print(f"The result is statistically significant at the {alpha*100:.0f}% level.")
        if test_type == 'one-tailed':
            print("We can reject the null hypothesis and conclude that there is a significant degradation trend across the wind farms (the true mean trend is likely less than zero).")
        else:
            print("We can reject the null hypothesis and conclude that the mean degradation trend is significantly different from zero.")
    else:
        print(f"The result is not statistically significant at the {alpha*100:.0f}% level.")
        print("We fail to reject the null hypothesis. There is not enough statistical evidence to conclude that a significant degradation trend exists across the wind farms.")

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns

# Create Q-Q plot to assess normality of the trend data
plt.figure(figsize=(12, 8), dpi=100)

# Create Q-Q plot using scipy
stats.probplot(trends_data, dist="norm", plot=plt)

# Customize the plot
plt.title('Q-Q Plot: Trend Data vs Normal Distribution', fontsize=16)
plt.xlabel('Theoretical Quantiles (Normal Distribution)', fontsize=12)
plt.ylabel('Sample Quantiles (Trend Data)', fontsize=12)
plt.grid(True, alpha=0.3)

# Add some statistics to the plot
mean_trend = np.mean(trends_data)
std_trend = np.std(trends_data, ddof=1)
n_samples = len(trends_data)

# Add text box with statistics
textstr = f'n = {n_samples}\nMean = {mean_trend:.3f}\nStd = {std_trend:.3f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

# Perform Shapiro-Wilk test for normality
shapiro_stat, shapiro_p = stats.shapiro(trends_data)

print("\n--- Normality Test Results ---")
print(f"Shapiro-Wilk Test:")
print(f"  Statistic: {shapiro_stat:.6f}")
print(f"  P-value: {shapiro_p:.6f}")

if shapiro_p > 0.05:
    print("  Result: Data appears to be normally distributed (p > 0.05)")
else:
    print("  Result: Data does not appear to be normally distributed (p ≤ 0.05)")

# Additional normality tests
anderson_stat, anderson_critical, anderson_significance = stats.anderson(trends_data, dist='norm')

print(f"\nAnderson-Darling Test:")
print(f"  Statistic: {anderson_stat:.6f}")
print(f"  Critical Values: {anderson_critical}")
print(f"  Significance Levels: {anderson_significance}%")

# Check if the statistic exceeds critical values
for i, (crit_val, sig_level) in enumerate(zip(anderson_critical, anderson_significance)):
    if anderson_stat > crit_val:
        print(f"  At {sig_level}% significance level: Reject normality")
    else:
        print(f"  At {sig_level}% significance level: Fail to reject normality")
        break

In [ ]:
import math
from scipy import stats

def calculate_sample_size(population_size, margin_of_error, confidence_level=0.95, population_proportion=0.5):
    # 1. Calculate the Z-score from the confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # 2. Calculate the sample size for an infinite population (Cochran's formula)
    n_0 = (z_score**2 * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # 3. Adjust the sample size for a finite population
    n = n_0 / (1 + (n_0 - 1) / population_size)

    return math.ceil(n)

POPULATION = 297

# Scenario 1: 95% confidence, 10% margin of error
confidence_95 = 0.95
margin_error_10 = 0.10
sample_size_10_pct = calculate_sample_size(POPULATION, margin_error_10, confidence_95)
print(f"For a population of {POPULATION}:")
print(f"Required sample size for 95% confidence and 10% margin of error: {sample_size_10_pct}")

# Scenario 2: 95% confidence, 5% margin of error
margin_error_5 = 0.05
sample_size_5_pct = calculate_sample_size(POPULATION, margin_error_5, confidence_95)
print(f"Required sample size for 95% confidence and 5% margin of error: {sample_size_5_pct}")

# Scenario 3: 99% confidence, 5% margin of error
confidence_99 = 0.99
sample_size_99_5_pct = calculate_sample_size(POPULATION, margin_error_5, confidence_99)
print(f"Required sample size for 99% confidence and 5% margin of error: {sample_size_99_5_pct}")

In [ ]:
import numpy as np
from scipy import stats

# Define the confidence level
confidence_level = 0.95

# Calculate the necessary parameters from your data
degrees_freedom = len(trends_data) - 1
sample_mean = np.mean(trends_data)
sample_standard_error = stats.sem(trends_data) # sem = Standard Error of the Mean

# Calculate the confidence interval using scipy.stats
# This function returns the lower and upper bounds of the interval
ci = stats.t.interval(confidence_level,
                        df=degrees_freedom,
                        loc=sample_mean,
                        scale=sample_standard_error)

# --- Print the results ---
print("--- 95% Confidence Interval for the Mean Trend ---")
print(f"Sample Mean: {sample_mean:.4f}% per year")
print(f"95% Confidence Interval (CI): [{ci[0]:.4f}, {ci[1]:.4f}]")

print("\nInterpretation:")
print(f"We are 95% confident that the true average annual degradation trend for the entire population of Turkish wind farms falls between {ci[0]:.2f}% and {ci[1]:.2f}%.")
if ci[1] < 0:
    print("Since the entire interval is below zero, this provides strong evidence that a statistically significant degradation is occurring.")

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# --- Variables to control selective plotting ---
plot_kde = True
plot_normal_fit = True
# ---------------------------------------------

# Plot the distribution of trends
plt.figure(figsize=(20, 10), dpi=100)

# Plot the histogram bars first. We use label='_nolegend_' to hide it from the legend,
# as the bars are self-explanatory.
ax = sns.histplot(data=trends, x='trend', bins=20, stat="density", label='_nolegend_')

# Get the data for fitting
data_for_fitting = trends['trend'].dropna()

# Plot KDE (Kernel Density Estimate) (Selective)
# This plots the smooth line over the histogram and gives it a clear label.
if plot_kde:
    sns.kdeplot(data=trends, x='trend', color='darkblue', ax=ax, label='KDE')

# Plot Normal Distribution Fit (Selective)
if plot_normal_fit:
    # Fit a normal distribution to the data
    mu, std = stats.norm.fit(data_for_fitting)

    # Generate x values for the PDF plot
    xmin, xmax = plt.xlim()
    x_norm = np.linspace(xmin, xmax, 100)

    # Calculate PDF of the normal distribution
    p_norm = stats.norm.pdf(x_norm, mu, std)

    # Plot the normal fit curve with a descriptive label
    plt.plot(x_norm, p_norm, 'k', linewidth=2, label=f'Normal Fit (μ={mu:.2f}, σ={std:.2f})')

# --- Plot Finalization ---
plt.title('Distribution of Capacity Factor Trends', fontsize=16)
plt.xlabel('Annual Trend (%/year)', fontsize=12)
plt.ylabel('Density', fontsize=12)

# Add a vertical line for the mean trend
plt.axvline(data_for_fitting.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean Trend ({data_for_fitting.mean():.2f})')
# Add a vertical line for the zero (no trend) point
plt.axvline(0, color='black', linestyle='-', linewidth=2, label='No Trend')
# -----------------

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
import pandas as pd
from src.DatabaseGetInfo import DatabaseAnalyzer
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from itertools import combinations
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

def analyze_trend_by_turbine(
    db_path,
    trends_csv_path,
    single_model_only=False,
    plot_type="boxplot",
    sort_by="trend_desc",
    group_by="model",
):
    """
    Analyzes and visualizes wind farm capacity factor trends grouped by turbine model or brand.

    FINAL CORRECTED LOGIC: This version correctly handles mixed-brand/mixed-model farms
    when single_model_only=False by "exploding" the data, ensuring all brands/models
    are represented in the plot. It also fixes a NameError from the previous version.

    Args:
        db_path (str): The absolute path to the project's SQLite database file (wfd.db).
        trends_csv_path (str): The path to the CSV file containing the trend data for each wind farm.
        single_model_only (bool): If True, analysis is restricted to single-model/single-brand farms.
        plot_type (str): The type of plot to generate, either 'boxplot' or 'barplot'.
        sort_by (str): Determines the sorting order of the categories on the x-axis of the plot.
        group_by (str): The characteristic to group by, either 'model' or 'brand'.

    Returns:
        pd.DataFrame or None: A DataFrame containing aggregated trend statistics.
    """
    # --- 1. Initial Setup and Data Loading ---
    analyzer = DatabaseAnalyzer.WindFarmAnalyzer(db_path)
    try:
        trends = pd.read_csv(trends_csv_path)
        trends = trends.dropna(subset=["wf_id", "trend"])
        trends["wf_id"] = trends["wf_id"].astype(int)
    except FileNotFoundError:
        print(f"Error: The trends CSV file was not found at {trends_csv_path}")
        return None

    wf_turbines_list = []
    for wf_id in trends["wf_id"]:
        try:
            temp_df = analyzer.get_wf_turbines_data(wf_id)
            if temp_df is not None and not temp_df.empty:
                temp_df["wf_id"] = wf_id
                wf_turbines_list.append(temp_df)
        except Exception as e:
            print(f"Could not retrieve turbine data for wf_id {wf_id}: {e}")
    if not wf_turbines_list:
        print("No valid turbine data could be fetched.")
        return None
    wf_turbines = pd.concat(wf_turbines_list, ignore_index=True)

    # --- 2. Data Preparation based on Analysis Type (DEFINITIVE LOGIC) ---
    group_col = "turbine_model" if group_by == "model" else "turbine_brand"

    # ** BUG FIX: Define display_name here, before it is used **
    display_name_map = {'model': 'Model', 'brand': 'Brand'}
    display_name = display_name_map.get(group_by, group_by.replace('_', ' ').title())

    filter_info_text = ""

    if single_model_only:
        # Strict filtering for 'pure' farms (single model or single brand)
        counts = wf_turbines.groupby("wf_id")[group_col].nunique()
        single_ids = counts[counts == 1].index.tolist()

        filtered_turbines = wf_turbines[wf_turbines["wf_id"].isin(single_ids)]
        merged_data = pd.merge(trends, filtered_turbines, on="wf_id", how="inner")

        # Create representative data (1 row per farm)
        plot_data = merged_data.groupby('wf_id').agg(
            trend=('trend', 'first'),
            **{group_col: (group_col, 'first')}
        ).reset_index()
        filter_info_text = f" (Single-{display_name} Farms Only)"

    else:
        # "All Farms" analysis, correctly handling mixed farms
        groups_per_farm = wf_turbines.groupby('wf_id')[group_col].unique().explode()
        plot_data = pd.merge(trends, groups_per_farm, on='wf_id', how="inner")
        filter_info_text = " (All Farms, including mixed)"
        print("NOTE: In 'All Farms' mode, a single farm's trend may be counted in multiple groups if it contains multiple brands/models.")

    if plot_data.empty:
        print("No data available after preparing data for plotting.")
        return None

    # --- 3. Grouping and Sorting ---
    grouped_trends = (
        plot_data.groupby(group_col)["trend"]
        .agg(["mean", "std", "count"])
        .reset_index()
    )
    if sort_by == "alphabetical": grouped_trends = grouped_trends.sort_values(by=group_col)
    elif sort_by == "trend_desc": grouped_trends = grouped_trends.sort_values(by="mean", ascending=False)
    elif sort_by == "trend_asc": grouped_trends = grouped_trends.sort_values(by="mean", ascending=True)
    elif sort_by == "count": grouped_trends = grouped_trends.sort_values(by="count", ascending=False)

    print(f"\nGrouped Trends by Turbine {display_name}{filter_info_text}:\n", grouped_trends)

    # --- 4. Visualization ---
    fig, ax = plt.subplots(figsize=(12, 8))

    sort_by_map = {
        'trend_desc': 'Sorted by Descending Mean Trend', 'trend_asc': 'Sorted by Ascending Mean Trend',
        'count': 'Sorted by Farm Count', 'alphabetical': 'Sorted Alphabetically'
    }
    sort_info_text = sort_by_map.get(sort_by, f"Sorted by {sort_by}")

    labels_with_counts = [f"{row[group_col]} (n={row['count']})" for _, row in grouped_trends.iterrows()]

    if plot_type == "boxplot":
        sns.boxplot(
            x=group_col, y="trend", data=plot_data,
            order=grouped_trends[group_col], ax=ax, showmeans=True,
            meanprops={"marker": "^", "markerfacecolor": "white", "markeredgecolor": "black", "markersize": "8"},
            whiskerprops={'color': 'black', 'linewidth': 1},
        )
        full_title = f"Trend Distribution by Turbine {display_name}{filter_info_text}\n({sort_info_text})"
        ax.set_title(full_title, fontsize=16)

        legend_elements = [
            Patch(facecolor='C0', alpha=0.4, edgecolor='C0', label='Interquartile Range (IQR: 25%-75%)'),
            Line2D([0], [0], color='black', lw=1, label='Whiskers (Range within 1.5*IQR)'),
            Line2D([0], [0], color=ax.lines[4].get_color(), lw=2, label='Median'),
            Line2D([0], [0], marker='^', color='w', label='Mean', markerfacecolor='white', markeredgecolor='black', markersize=10)
        ]
        ax.legend(handles=legend_elements, loc='best', frameon=True)

    elif plot_type == "barplot":
        bars = ax.bar(grouped_trends[group_col], grouped_trends["mean"], yerr=grouped_trends["std"], capsize=5)
        full_title = f"Mean Trend by Turbine {display_name}{filter_info_text}\n({sort_info_text})"
        ax.set_title(full_title, fontsize=16)
        for bar in bars:
            yval = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2.0, yval, f"{yval:.2f}", va="bottom" if yval >= 0 else "top", ha="center", fontsize=9)

    ax.set_xticks(range(len(labels_with_counts)))
    ax.set_xticklabels(labels_with_counts, rotation=90, ha='right')
    ax.set_xlabel(f"Turbine {display_name}", fontsize=12)
    ax.set_ylabel("Trend (%/year)", fontsize=12)

    ymin, ymax = ax.get_ylim()
    tick_min = np.floor(ymin / 0.25) * 0.25
    tick_max = np.ceil(ymax / 0.25) * 0.25
    ax.set_yticks(np.arange(tick_min, tick_max + 0.25, 0.25))

    ax.axhline(0, color='grey', linestyle='--', linewidth=0.8)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    fig.tight_layout()
    plt.savefig("figures_for_thesis/trends_for_{}_by_turbine_{}.png".format(display_name, group_by), dpi=100)
    plt.show()

    return grouped_trends

In [ ]:
# Example Usage
db_path = "/home/wheatley/WFD/wfd.db"  # Replace with your database path
trends_csv_path = "trends.csv"

# Run for all farms
all_farms_results = analyze_trend_by_turbine(db_path, trends_csv_path, single_model_only=False, sort_by="trend_desc") # "alphabetical" "trend_asc" "trend_desc" "count"

In [ ]:
single_model_results = analyze_trend_by_turbine(db_path, trends_csv_path, single_model_only=True, sort_by="trend_desc")

In [ ]:
model_results = analyze_trend_by_turbine(db_path, trends_csv_path, single_model_only=True, group_by='brand', sort_by='trend_desc')

In [ ]:
import pandas as pd
from src.DatabaseGetInfo import DatabaseAnalyzer
import os
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

def analyze_trend_by_elevation(db_path, trends_csv_path):
    """
    Analyzes and plots the relationship between wind farm elevation and capacity factor trends.

    Args:
        db_path (str): Path to the SQLite database.
        trends_csv_path (str): Path to the CSV file containing trend data.

    Returns:
        pd.DataFrame: A dataframe containing the correlation matrix between trend and elevation.
    """
    # --- Initial Setup ---
    analyzer = DatabaseAnalyzer.WindFarmAnalyzer(db_path)
    trends = pd.read_csv(trends_csv_path)
    trends = trends.dropna(subset=['trend'])
    trends['wf_id'] = trends['wf_id'].astype(int)

    # --- Fetch and Process Elevation Data ---
    elevations = []
    for wf_id in trends['wf_id']:
        try:
            el = analyzer.find_elevations(wf_id)
            if el:
                # Calculate average elevation in meters and then convert to kilometers
                avg_elevation_meters = sum(x[3] for x in el) / len(el)
                elevations.append(avg_elevation_meters / 1000.0) # Convert to km
            else:
                elevations.append(None)
        except (KeyError, IndexError):
            # Handle cases where wf_id is not found or elevation data is malformed
            elevations.append(None)

    trends['elevation'] = elevations

    # Drop rows where elevation could not be determined to ensure clean calculations
    trends.dropna(subset=['elevation'], inplace=True)

    # --- Plotting ---
    # UPDATED: Set figure size to match other plots
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create scatter plot
    ax.scatter(trends['elevation'], trends['trend'], label='WPP Data Points', alpha=0.7, zorder=5)

    # Calculate and plot the trendline
    if not trends.empty:
        # NOTE: r_value is no longer needed for the label
        slope, intercept, _, _, _ = stats.linregress(trends['elevation'], trends['trend'])

        # UPDATED: Simplified the legend label to remove R-squared
        formula_label = f'Trendline (y = {slope:.4f}x + {intercept:.2f})'

        ax.plot(trends['elevation'], slope * trends['elevation'] + intercept,
                color='red', label=formula_label, zorder=10)

    # Set a clean, consistent title
    ax.set_title('Capacity Factor Trend vs. Mean Site Elevation', fontsize=16)

    # Set consistent labels and fonts
    ax.set_xlabel('Mean Site Elevation (km)', fontsize=12)
    ax.set_ylabel('Trend (%/year)', fontsize=12)

    # Set y-axis ticks to increments of 0.25
    ymin, ymax = ax.get_ylim()
    tick_min = np.floor(ymin / 0.25) * 0.25
    tick_max = np.ceil(ymax / 0.25) * 0.25
    ax.set_yticks(np.arange(tick_min, tick_max + 0.25, 0.25))

    # Add grid and reference line for consistency
    ax.axhline(0, color='grey', linestyle='--', linewidth=0.8)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Display legend
    ax.legend(frameon=True, loc='best')

    fig.tight_layout()
    plt.savefig("figures_for_thesis/Capacity trend by Elevation (by Trends)')")
    plt.show()

    # Return the correlation matrix for verification
    correlation_matrix = trends[['trend', 'elevation']].corr()
    print("Correlation Matrix:")
    print(correlation_matrix)

    return correlation_matrix

# --- How to use the new function ---
# Define your database and trends file paths
db_path = os.path.abspath('/home/wheatley/WFD/wfd.db')
trends_csv_path = 'trends.csv'

# Call the function to generate the plot and get the correlation
correlation_result = analyze_trend_by_elevation(db_path, trends_csv_path)

In [ ]:
import pandas as pd
from src.DatabaseGetInfo import DatabaseAnalyzer
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

def analyze_turbine_trends_by_year(
    db_path,
    trends_csv_path,
    single_model_only=False,
    sort_by="year_asc",
):
    """
    Analyzes and visualizes wind farm capacity factor trends grouped by installation year.
    (Args and initial logic remain the same)
    """
    # --- 1. Initial Setup and Data Loading ---
    analyzer = DatabaseAnalyzer.WindFarmAnalyzer(db_path)
    try:
        trends = pd.read_csv(trends_csv_path)
        trends = trends.dropna(subset=["wf_id", "trend"])
        trends["wf_id"] = trends["wf_id"].astype(int)
    except FileNotFoundError:
        print(f"Error: The trends CSV file was not found at {trends_csv_path}")
        return None

    wf_turbines_list = []
    for wf_id in trends["wf_id"]:
        try:
            temp_df = analyzer.get_wf_turbines_data(wf_id)
            if temp_df is not None and not temp_df.empty:
                temp_df["start_date_of_operation"] = pd.to_datetime(
                    temp_df["start_date_of_operation"], errors="coerce"
                )
                temp_df.dropna(subset=["start_date_of_operation"], inplace=True)
                if not temp_df.empty:
                    temp_df["wf_id"] = wf_id
                    wf_turbines_list.append(temp_df)
        except Exception as e:
            print(f"Could not retrieve or process turbine data for wf_id {wf_id}: {e}")
            continue

    if not wf_turbines_list:
        print("No valid turbine data could be fetched from the database.")
        return None
    wf_turbines = pd.concat(wf_turbines_list, ignore_index=True)

    # --- 2. Conditional Filtering ---
    if single_model_only:
        # ... (This logic is unchanged) ...
        def check_homogeneity(farm_data):
            is_single_model = farm_data["turbine_model"].nunique() == 1
            is_single_year = farm_data["start_date_of_operation"].dt.year.nunique() == 1
            return is_single_model and is_single_year
        homogeneous_wf_ids = [
            wf_id for wf_id, group_df in wf_turbines.groupby("wf_id") if check_homogeneity(group_df)
        ]
        if not homogeneous_wf_ids:
            print("No wind farms met the homogeneity criteria.")
            return None
        wf_turbines = wf_turbines[wf_turbines["wf_id"].isin(homogeneous_wf_ids)]

    # --- 3. Merging and Preparing Representative Data ---
    # ... (This logic is unchanged) ...
    merged_data = pd.merge(trends, wf_turbines, on="wf_id", how="inner")
    if merged_data.empty:
        print("No data available after merging.")
        return None
    representative_data = merged_data.groupby("wf_id").agg(
        trend=("trend", "first"),
        installation_year=("start_date_of_operation", lambda x: x.dt.year.min()),
    ).reset_index()
    if representative_data.empty:
        print("Data became empty after creating the representative dataset.")
        return None

    # --- 4. Grouping and Sorting ---
    # ... (This logic is unchanged) ...
    grouped_trends = (
        representative_data.groupby("installation_year")["trend"]
        .agg(["mean", "std", "count"]).reset_index()
    )
    if sort_by == "year_asc":
        grouped_trends = grouped_trends.sort_values(by="installation_year", ascending=True)
    elif sort_by == "trend_desc":
        grouped_trends = grouped_trends.sort_values(by="mean", ascending=False)
    elif sort_by == "trend_asc":
        grouped_trends = grouped_trends.sort_values(by="mean", ascending=True)
    elif sort_by == "count":
        grouped_trends = grouped_trends.sort_values(by="count", ascending=False)
    else:
        grouped_trends = grouped_trends.sort_values(by="installation_year", ascending=True)

    print("\nGrouped Trends by Installation Year:\n", grouped_trends)

    # --- 5. Visualization (UPDATED) ---
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create descriptive text for sorting and filtering
    sort_by_map = {
        'year_asc': 'Sorted by Year',
        'trend_desc': 'Sorted by Descending Mean Trend',
        'trend_asc': 'Sorted by Ascending Mean Trend',
        'count': 'Sorted by Data Count'
    }
    sort_info_text = sort_by_map.get(sort_by, f"Sorted by {sort_by}")
    filter_info_text = " (Homogeneous Farms Only)" if single_model_only else " (All Farms)"

    # Create labels with data counts for the x-axis
    labels_with_counts = [
        f"{int(row['installation_year'])} (n={int(row['count'])})" for _, row in grouped_trends.iterrows()
    ]

    # Create the boxplot with consistent styling
    sns.boxplot(
        x="installation_year", y="trend", data=representative_data,
        order=grouped_trends["installation_year"], ax=ax, showmeans=True,
        meanprops={"marker": "^", "markerfacecolor": "white", "markeredgecolor": "black", "markersize": "8"},
        whiskerprops={'color': 'black', 'linewidth': 1},
    )

    # Set the full, descriptive title
    full_title = f"Distribution of CF Trends by Installation Year{filter_info_text}\n({sort_info_text})"
    ax.set_title(full_title, fontsize=16)

    # Add the detailed legend explaining the boxplot components
    legend_elements = [
        Patch(facecolor='C0', alpha=0.4, edgecolor='C0', label='Interquartile Range (IQR: 25%-75%)'),
        Line2D([0], [0], color='black', lw=1, label='Whiskers (Range within 1.5*IQR)'),
        Line2D([0], [0], color=ax.lines[4].get_color(), lw=2, label='Median'),
        Line2D([0], [0], marker='^', color='w', label='Mean',
               markerfacecolor='white', markeredgecolor='black', markersize=10)
    ]
    ax.legend(handles=legend_elements, loc='best', frameon=True)

    # Apply consistent axis labels and ticks
    ax.set_xticks(range(len(labels_with_counts)))
    ax.set_xticklabels(labels_with_counts, rotation=45, ha="right")
    ax.set_xlabel("Installation Year", fontsize=12)
    ax.set_ylabel("Capacity Factor Trend (%/year)", fontsize=12)

    ymin, ymax = ax.get_ylim()
    tick_min = np.floor(ymin / 0.25) * 0.25
    tick_max = np.ceil(ymax / 0.25) * 0.25
    ax.set_yticks(np.arange(tick_min, tick_max + 0.25, 0.25))

    ax.axhline(0, color='grey', linestyle='--', linewidth=0.8)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    fig.tight_layout()
    fig.savefig("figures_for_thesis/Capacity Trend by Installation Year (by Trends) (single modal={single_model_only})".format(single_model_only=single_model_only),dpi=100)
    plt.show()

    return grouped_trends


db_path = os.path.abspath('/home/wheatley/WFD/wfd.db')
trends_csv_path = 'trends.csv'

results_all_farms = analyze_turbine_trends_by_year(
    db_path,
    trends_csv_path,
    single_model_only=True,
    sort_by="year_asc",
)

In [ ]:
results_all_farms = analyze_turbine_trends_by_year(
    db_path,
    trends_csv_path,
    single_model_only=False,
    sort_by="year_asc",
)

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from shapely import wkb # For Well-Known Binary
import numpy as np

def analyze_farm_size_layout_correlations(db_path, trends_csv_path):
    """
    Analyzes and visualizes correlations between CF trends and
    farm size/layout characteristics (number of turbines, total capacity, capacity density).
    """
    try:
        # --- Data Loading and SpatiaLite Connection (Unchanged) ---
        trends_df = pd.read_csv(trends_csv_path)
        trends_df = trends_df.dropna(subset=['wf_id', 'trend'])
        trends_df['wf_id'] = trends_df['wf_id'].astype(int)

        conn = sqlite3.connect(db_path)
        conn.enable_load_extension(True)
        try:
            conn.execute("SELECT load_extension('mod_spatialite')")
        except sqlite3.OperationalError:
            try:
                conn.execute("SELECT load_extension('/usr/lib/x86_64-linux-gnu/mod_spatialite.so')")
            except sqlite3.OperationalError as e:
                print(f"Warning: Could not load SpatiaLite extension. Spatial functions might fail. Error: {e}")

        # --- Data Querying and Processing (Unchanged) ---
        query_turbines = "SELECT wf_id, SUM(turbine_number) as num_turbines FROM wf_turbines GROUP BY wf_id"
        num_turbines_df = pd.read_sql_query(query_turbines, conn)
        if not num_turbines_df.empty:
            num_turbines_df['wf_id'] = pd.to_numeric(num_turbines_df['wf_id'], errors='coerce').astype('Int64')

        query_wf_info = "SELECT wf_id, installed_power_electrical, ST_AsBinary(farm_boundary_geom) AS farm_boundary_wkb FROM wf"
        wf_info_df = pd.read_sql_query(query_wf_info, conn)
        if not wf_info_df.empty:
            wf_info_df['wf_id'] = pd.to_numeric(wf_info_df['wf_id'], errors='coerce').astype('Int64')

        conn.close()

        merged_df = pd.merge(trends_df, num_turbines_df, on='wf_id', how='inner')
        merged_df = pd.merge(merged_df, wf_info_df, on='wf_id', how='inner')

        if merged_df.empty:
            print("Merging resulted in an empty DataFrame.")
            return None

        def calculate_area_from_standard_wkb(wkb_input):
            if pd.isna(wkb_input) or wkb_input is None: return np.nan
            bytes_to_parse = wkb_input if isinstance(wkb_input, bytes) else bytes.fromhex(wkb_input)
            try:
                return wkb.loads(bytes_to_parse).area
            except Exception:
                return np.nan

        merged_df['farm_area_sq_m'] = merged_df['farm_boundary_wkb'].apply(calculate_area_from_standard_wkb)
        merged_df['farm_area_sq_km'] = merged_df['farm_area_sq_m'] / 1_000_000
        valid_area_mask = merged_df['farm_area_sq_km'].notna() & (merged_df['farm_area_sq_km'] > 1e-9)
        merged_df.loc[valid_area_mask, 'capacity_density_MWe_per_sq_km'] = merged_df.loc[valid_area_mask, 'installed_power_electrical'] / merged_df.loc[valid_area_mask, 'farm_area_sq_km']

        # --- Create Scatter Plots (UPDATED) ---
        metrics_to_plot = {
            'num_turbines': 'Number of Turbines',
            'installed_power_electrical': 'Total Installed Capacity (MWe)',
            'capacity_density_MWe_per_sq_km': 'Capacity Density (MWe/km²)'
        }

        for col, name in metrics_to_plot.items():
            plot_df = merged_df.dropna(subset=['trend', col])
            if plot_df.empty or len(plot_df) < 2:
                print(f"Skipping plot for {name} due to insufficient valid data.")
                continue

            # Create a separate, styled figure for each metric
            fig, ax = plt.subplots(figsize=(12, 8))

            # 1. Plot the scatter points
            ax.scatter(x=plot_df[col], y=plot_df['trend'], label='WPP Data Points', alpha=0.7)

            # 2. Calculate and plot the regression line
            slope, intercept, _, _, _ = stats.linregress(plot_df[col], plot_df['trend'])
            formula_label = f'Trendline (y = {slope:.4f}x + {intercept:.2f})'

            # Create a smooth line for plotting
            x_vals = np.linspace(plot_df[col].min(), plot_df[col].max(), 100)
            y_vals = slope * x_vals + intercept
            ax.plot(x_vals, y_vals, color='red', label=formula_label)

            # 3. Apply all consistent styling elements
            ax.set_title(f'Capacity Factor Trend vs. {name}', fontsize=16)
            ax.set_xlabel(name, fontsize=12)
            ax.set_ylabel('Trend (%/year)', fontsize=12)

            ymin, ymax = ax.get_ylim()
            tick_min = np.floor(ymin / 0.25) * 0.25
            tick_max = np.ceil(ymax / 0.25) * 0.25
            ax.set_yticks(np.arange(tick_min, tick_max + 0.25, 0.25))

            ax.axhline(0, color='grey', linestyle='--', linewidth=0.8)
            ax.grid(axis='y', linestyle='--', alpha=0.7)

            ax.legend(frameon=True, loc='best')
            fig.tight_layout()
            plt.savefig("figures_for_thesis/Capacity trend by {name} (by Trends)".format(name=col), dpi=100)
            plt.show()

        return merged_df

    except Exception as e:
        print(f"An error occurred in analyze_farm_size_layout_correlations: {e}")
        import traceback
        traceback.print_exc()
        return None

# --- Example Usage ---
db_path = "/home/wheatley/WFD/wfd.db"
trends_csv_path = "trends.csv"
farm_size_layout_df = analyze_farm_size_layout_correlations(db_path, trends_csv_path)

In [ ]:
# Import necessary libraries for analysis and display
import pandas as pd
import numpy as np
import sqlite3
from scipy import stats
from itertools import combinations
from IPython.display import display, Markdown
from src.DatabaseGetInfo import DatabaseAnalyzer

# ==============================================================================
# DATA PREPARATION FOR ALL ANALYSES
# ==============================================================================
display(Markdown("## Final Summary Tables & Statistical Tests"))
display(Markdown("This section consolidates the key findings. Each analysis is performed on the most appropriate subset of the data to ensure results are scientifically robust and clear."))

# --- 1. Data Consolidation ---
print("--- Consolidating all data for summary tables... ---")
db_path = "/home/wheatley/WFD/wfd.db"
trends_csv_path = "trends.csv"

# Load trends data
trends_df = pd.read_csv(trends_csv_path)
trends_df = trends_df.dropna(subset=['wf_id', 'trend'])
trends_df['wf_id'] = trends_df['wf_id'].astype(int)

# ... (Database connection and data loading logic remains the same) ...
analyzer = DatabaseAnalyzer.WindFarmAnalyzer(db_path)
conn = sqlite3.connect(db_path)
conn.enable_load_extension(True)
try:
    conn.execute("SELECT load_extension('mod_spatialite')")
except sqlite3.OperationalError:
    try: conn.execute("SELECT load_extension('/usr/lib/x86_64-linux-gnu/mod_spatialite.so')")
    except Exception as e: print(f"Warning: SpatiaLite load failed: {e}")

num_turbines_df = pd.read_sql_query("SELECT wf_id, SUM(turbine_number) as num_turbines FROM wf_turbines GROUP BY wf_id", conn)
wf_turbines_details_df = pd.read_sql_query("SELECT wf_id, turbine_brand, turbine_model, start_date_of_operation FROM wf_turbines", conn)
conn.close()

# Merge all data sources into a detailed dataframe
summary_df_full_details = pd.merge(trends_df, num_turbines_df, on='wf_id', how='left')
summary_df_full_details = pd.merge(summary_df_full_details, wf_turbines_details_df, on='wf_id', how='left')
summary_df_full_details['start_date_of_operation'] = pd.to_datetime(summary_df_full_details['start_date_of_operation'], errors='coerce')

# --- 2. Create Purpose-Built, Filtered Datasets ---
print("--- Preparing filtered datasets for specific analyses... ---")

# Base representative dataset (one row per farm)
representative_df_base = summary_df_full_details.groupby('wf_id').agg(
    trend=('trend', 'first')
).reset_index()

# Filter 1: Single-Brand Farms
single_brand_wf_ids = [wf_id for wf_id, group_df in summary_df_full_details.groupby("wf_id") if group_df["turbine_brand"].nunique() == 1]
representative_df_single_brand = summary_df_full_details[summary_df_full_details['wf_id'].isin(single_brand_wf_ids)].groupby('wf_id').agg(
    trend=('trend', 'first'),
    turbine_brand=('turbine_brand', 'first')
).reset_index()

# Filter 2: Single-Model Farms
single_model_wf_ids = [wf_id for wf_id, group_df in summary_df_full_details.groupby("wf_id") if group_df["turbine_model"].nunique() == 1]
representative_df_single_model = summary_df_full_details[summary_df_full_details['wf_id'].isin(single_model_wf_ids)].groupby('wf_id').agg(
    trend=('trend', 'first'),
    turbine_model=('turbine_model', 'first')
).reset_index()

# Filter 3: Homogeneous Farms
def check_homogeneity(farm_data):
    return farm_data["turbine_model"].nunique() == 1 and farm_data["start_date_of_operation"].dt.year.nunique() == 1
homogeneous_wf_ids = [wf_id for wf_id, group_df in summary_df_full_details.groupby("wf_id") if check_homogeneity(group_df)]
representative_df_homogeneous = summary_df_full_details[summary_df_full_details['wf_id'].isin(homogeneous_wf_ids)].groupby('wf_id').agg(
    trend=('trend', 'first'),
    # *** THIS IS THE FIX: Directly aggregate the year component ***
    installation_year=('start_date_of_operation', lambda x: x.dt.year.min())
).reset_index()

print(f"\nTotal unique farms in dataset: {len(representative_df_base)}")
print(f"Number of Single-Brand farms: {len(representative_df_single_brand)}")
print(f"Number of Single-Model farms: {len(representative_df_single_model)}")
print(f"Number of Homogeneous farms: {len(representative_df_homogeneous)}")
print("--- Generating final tables. ---\n")


# ==============================================================================
# SUMMARY TABLES
# ==============================================================================

# --- Table 1: Trend Distribution by Turbine Brand (Single-Brand Farms) ---
display(Markdown("### Summary Table 1: Trend Distribution by Turbine Brand (Single-Brand Farms)"))
brand_summary_table = representative_df_single_brand.groupby('turbine_brand')['trend'].agg(['count', 'mean', 'median']).reset_index()
brand_summary_table = brand_summary_table.sort_values(by='count', ascending=False).reset_index(drop=True)
display(brand_summary_table)

# --- Table 2: Trend Distribution by Turbine Model (Single-Model Farms) ---
display(Markdown("\n### Summary Table 2: Trend Distribution by Turbine Model (Single-Model Farms)"))
model_summary_table = representative_df_single_model.groupby('turbine_model')['trend'].agg(['count', 'mean', 'median']).reset_index()
model_summary_table = model_summary_table.sort_values(by='count', ascending=False).reset_index(drop=True)
display(model_summary_table)

# --- Table 3: Trend Distribution by Installation Year (Homogeneous Farms) ---
display(Markdown("\n### Summary Table 3: Trend Distribution by Installation Year (Homogeneous Farms)"))
year_summary_table = representative_df_homogeneous.groupby('installation_year')['trend'].agg(['count', 'mean', 'median']).reset_index()
year_summary_table['installation_year'] = year_summary_table['installation_year'].astype(int)
year_summary_table = year_summary_table.sort_values(by='installation_year', ascending=True).reset_index(drop=True)
display(year_summary_table)

In [ ]:
# Import necessary libraries for analysis and display
import pandas as pd
import numpy as np
import sqlite3
from scipy import stats
from itertools import combinations
from IPython.display import display, Markdown
from src.DatabaseGetInfo import DatabaseAnalyzer

# ==============================================================================
# DATA PREPARATION FOR ALL STATISTICAL TESTS
# ==============================================================================
display(Markdown("## Final Statistical Significance Tests"))
display(Markdown("This section performs quantitative statistical tests to validate the visual findings. Each test is run on the most appropriate dataset to ensure the conclusions are scientifically sound."))

# --- 1. Data Consolidation ---
print("--- Consolidating all data for tests... ---")
db_path = "/home/wheatley/WFD/wfd.db"
trends_csv_path = "trends.csv"

# Load trends data
trends_df = pd.read_csv(trends_csv_path)
trends_df = trends_df.dropna(subset=['wf_id', 'trend'])
trends_df['wf_id'] = trends_df['wf_id'].astype(int)

# ... (Database connection and data loading logic) ...
analyzer = DatabaseAnalyzer.WindFarmAnalyzer(db_path)
conn = sqlite3.connect(db_path)
conn.enable_load_extension(True)
try:
    conn.execute("SELECT load_extension('mod_spatialite')")
except sqlite3.OperationalError:
    try: conn.execute("SELECT load_extension('/usr/lib/x86_64-linux-gnu/mod_spatialite.so')")
    except Exception as e: print(f"Warning: SpatiaLite load failed: {e}")

elevations = []
for wf_id in trends_df['wf_id']:
    try:
        el = analyzer.find_elevations(wf_id)
        if el: elevations.append(sum(x[3] for x in el) / len(el) / 1000.0) # in km
        else: elevations.append(np.nan)
    except (KeyError, IndexError): elevations.append(np.nan)
trends_df['elevation'] = elevations

num_turbines_df = pd.read_sql_query("SELECT wf_id, SUM(turbine_number) as num_turbines FROM wf_turbines GROUP BY wf_id", conn)
wf_info_df = pd.read_sql_query("SELECT wf_id, installed_power_electrical, ST_AsBinary(farm_boundary_geom) AS farm_boundary_wkb FROM wf", conn)
wf_turbines_details_df = pd.read_sql_query("SELECT wf_id, turbine_brand, turbine_model, start_date_of_operation FROM wf_turbines", conn)
conn.close()

# Merge all data sources into a detailed dataframe
summary_df_full_details = pd.merge(trends_df, num_turbines_df, on='wf_id', how='left')
summary_df_full_details = pd.merge(summary_df_full_details, wf_info_df, on='wf_id', how='left')
summary_df_full_details = pd.merge(summary_df_full_details, wf_turbines_details_df, on='wf_id', how='left')
summary_df_full_details['start_date_of_operation'] = pd.to_datetime(summary_df_full_details['start_date_of_operation'], errors='coerce')

def calculate_area_from_standard_wkb(wkb_input):
    if pd.isna(wkb_input): return np.nan
    try: return wkb.loads(wkb_input if isinstance(wkb_input, bytes) else bytes.fromhex(wkb_input)).area
    except: return np.nan
summary_df_full_details['farm_area_sq_km'] = summary_df_full_details['farm_boundary_wkb'].apply(calculate_area_from_standard_wkb) / 1_000_000
valid_area_mask = summary_df_full_details['farm_area_sq_km'].notna() & (summary_df_full_details['farm_area_sq_km'] > 1e-9)
summary_df_full_details.loc[valid_area_mask, 'capacity_density_MWe_per_sq_km'] = summary_df_full_details.loc[valid_area_mask, 'installed_power_electrical'] / summary_df_full_details.loc[valid_area_mask, 'farm_area_sq_km']


# --- 2. Create Purpose-Built, Representative Datasets ---
print("--- Preparing representative datasets for specific analyses... ---")

# Dataset for Continuous Variable analysis (All Farms)
representative_df_all_farms = summary_df_full_details.groupby('wf_id').agg(
    trend=('trend', 'first'),
    elevation=('elevation', 'first'),
    num_turbines=('num_turbines', 'first'),
    installed_power_electrical=('installed_power_electrical', 'first'),
    capacity_density_MWe_per_sq_km=('capacity_density_MWe_per_sq_km', 'first')
).reset_index()

# Dataset for Brand analysis (Single-Brand Farms)
single_brand_wf_ids = [wf_id for wf_id, g in summary_df_full_details.groupby("wf_id") if g["turbine_brand"].nunique() == 1]
representative_df_single_brand = summary_df_full_details[summary_df_full_details['wf_id'].isin(single_brand_wf_ids)].groupby('wf_id').agg(
    trend=('trend', 'first'), turbine_brand=('turbine_brand', 'first')
).reset_index()

# Dataset for Year analysis (Homogeneous Farms)
def check_homogeneity(farm_data):
    return farm_data["turbine_model"].nunique() == 1 and farm_data["start_date_of_operation"].dt.year.nunique() == 1
homogeneous_wf_ids = [wf_id for wf_id, g in summary_df_full_details.groupby("wf_id") if check_homogeneity(g)]
representative_df_homogeneous = summary_df_full_details[summary_df_full_details['wf_id'].isin(homogeneous_wf_ids)].groupby('wf_id').agg(
    trend=('trend', 'first'), installation_year=('start_date_of_operation', lambda x: x.dt.year.min())
).reset_index()

print(f"\nTotal unique farms in dataset: {len(representative_df_all_farms)}")
print(f"Number of Single-Brand farms: {len(representative_df_single_brand)}")
print(f"Number of Homogeneous farms: {len(representative_df_homogeneous)}")
print("--- Running final statistical tests on appropriate datasets. ---\n")


# ==============================================================================
# STATISTICAL SIGNIFICANCE TESTS
# ==============================================================================

# --- Test A: Differences Among Turbine Brands (on Single-Brand Farms) ---
display(Markdown("### Test A: Significance of Differences Among Turbine Brands (Single-Brand Farms)"))
df_test_brand = representative_df_single_brand
brand_groups = [group['trend'].values for name, group in df_test_brand.groupby('turbine_brand')]
f_statistic, anova_p_value = stats.f_oneway(*brand_groups)
display(Markdown(f"**ANOVA Test Result:** F-statistic = {f_statistic:.4f}, P-value = {anova_p_value:.4f}"))
if anova_p_value > 0.05:
    print("Conclusion: The high p-value indicates no statistically significant evidence of a difference in mean trends among the brands.")

# --- Test B: Differences Among Installation Years (on Homogeneous Farms) ---
display(Markdown("\n### Test B: Significance of Differences Among Installation Years (Homogeneous Farms)"))
df_test_year = representative_df_homogeneous.dropna(subset=['installation_year'])
year_groups = [group['trend'].values for name, group in df_test_year.groupby('installation_year')]
if len(year_groups) > 1:
    f_statistic_yr, anova_p_value_yr = stats.f_oneway(*year_groups)
    display(Markdown(f"**ANOVA Test Result:** F-statistic = {f_statistic_yr:.4f}, P-value = {anova_p_value_yr:.4f}"))
    if anova_p_value_yr > 0.05:
        print("Conclusion: The high p-value indicates no statistical evidence of a difference in mean trends among the installation years.")
else:
    print("Not enough year diversity in the homogeneous dataset to perform ANOVA.")

# --- Test C: Differences for Binned Continuous Variables (on ALL FARMS) ---
display(Markdown("\n### Test C: Significance for Binned Continuous Variables (All Farms)"))
def perform_binned_ttest(df, metric_col, metric_name):
    df_filtered = df.dropna(subset=[metric_col])
    if len(df_filtered) < 4:
        print(f"\n--- {metric_name} ---\nNot enough data to perform test.")
        return
    threshold = df_filtered[metric_col].median()
    low_group = df_filtered[df_filtered[metric_col] < threshold]['trend']
    high_group = df_filtered[df_filtered[metric_col] >= threshold]['trend']
    if len(low_group) < 2 or len(high_group) < 2:
        print(f"\n--- {metric_name} ---\nNot enough data in one of the bins to perform test.")
        return
    t_stat, p_val = stats.ttest_ind(low_group, high_group, equal_var=False)
    print(f"\n--- {metric_name} ---")
    print(f"Binning Threshold: < {threshold:.2f} (Low, n={len(low_group)}) vs. >= {threshold:.2f} (High, n={len(high_group)})")
    print(f"  T-statistic: {t_stat:.3f}, P-value: {p_val:.3f}")
    if p_val < 0.05: print("  Result: Statistically significant difference found between Low and High groups.")
    else: print("  Result: No statistically significant difference between Low and High groups.")

df_test_continuous = representative_df_all_farms
perform_binned_ttest(df_test_continuous, 'elevation', 'Mean Site Elevation (km)')
perform_binned_ttest(df_test_continuous, 'num_turbines', 'Number of Turbines')
perform_binned_ttest(df_test_continuous, 'installed_power_electrical', 'Total Installed Capacity (MWe)')
perform_binned_ttest(df_test_continuous, 'capacity_density_MWe_per_sq_km', 'Capacity Density (MWe/km²)')